In [1]:
import os

In [2]:
%pwd

'c:\\Users\\lequy\\OneDrive\\Desktop\\chest-cancer-classifier\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\lequy\\OneDrive\\Desktop\\chest-cancer-classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from chestCancerClassifier.constants import *
from chestCancerClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf
import os
import shutil
from pathlib import Path

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def copy_relevant_classes(self, src_dir, dest_dir, classes):
        """
        Copy only the relevant classes from the source directory to the destination directory.

        Parameters:
        src_dir (str): Source directory containing the full dataset.
        dest_dir (str): Destination directory to contain only the relevant classes.
        classes (list): List of class names to be copied.
        """
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        for class_name in classes:
            src_class_dir = os.path.join(src_dir, class_name)
            dest_class_dir = os.path.join(dest_dir, class_name)

            if not os.path.exists(dest_class_dir):
                os.makedirs(dest_class_dir)

            # Copy all files from the source class directory to the destination class directory
            for filename in os.listdir(src_class_dir):
                src_file = os.path.join(src_class_dir, filename)
                dest_file = os.path.join(dest_class_dir, filename)
                shutil.copyfile(src_file, dest_file)

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params

        # Define source and destination directories
        src_dir = os.path.join(self.config.data_ingestion.unzip_dir, "Data", "test")
        dest_dir = os.path.join(self.config.data_ingestion.unzip_dir, "Data", "new")

        # List of relevant classes
        relevant_classes = ['adenocarcinoma', 'normal']

        # Copy relevant classes
        self.copy_relevant_classes(src_dir, dest_dir, relevant_classes)

        print("Relevant classes copied successfully!")
        
        training_data = dest_dir
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            class_mode='categorical',
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            class_mode='categorical',
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-07-08 18:30:16,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-08 18:30:16,015: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-08 18:30:16,016: INFO: common: created directory at: artifacts]
Relevant classes copied successfully!
[2024-07-08 18:30:16,102: INFO: common: created directory at: artifacts\training]
Found 34 images belonging to 2 classes.
Found 140 images belonging to 2 classes.
8/8 [==============================] - 8s 928ms/step - loss: 6.4339 - accuracy: 0.6935 - val_loss: 17.3570 - val_accuracy: 0.7500
